In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import pandasql as ps
from pandasql import sqldf

sns.set(font_scale=3)
sns.set(color_codes=True)
%matplotlib inline
import matplotlib.pyplot as plt


plt.rcParams["figure.figsize"] = [16, 16] 


pysqldf = lambda q: sqldf(q, globals())



In [ ]:
trans_df = pd.read_excel("../data/QuVa-GAIL DataRequest_4_5.xlsx","Daily Orders")
items_df = pd.read_excel("../data/QuVa-GAIL DataRequest_4_5.xlsx","Item Master")

In [ ]:
new_cols = [c.replace(" " , "_") for c in trans_df.columns]
new_cols
trans_df.columns = new_cols


trans_df.Order_Date = pd.to_datetime(trans_df.Order_Date)

In [ ]:
trans_df.columns

In [ ]:
trans_df.to_csv("quva_trans.csv",index=False)


In [ ]:
sns.distplot(trans_df.groupby("Customer_ID").count()[["Item_ID"]],kde=False)
plt.show()

In [ ]:

trans_ts = pysqldf("""SELECT strftime("%Y", Order_Date) year, 
strftime("%m", Order_Date) month,
strftime("%Y_%m", Order_Date) year_month, count(*) trans_count, 
count(distinct item_id) number_of_distinct_items,
count(distinct Customer_ID) number_of_customers,
sum(Daily_Revenue) total_revenue ,
sum(Daily_Revenue)/count(*) average_transaction_value
FROM trans_df group by strftime("%Y_%m", Order_Date),
strftime("%Y", Order_Date),strftime("%m", Order_Date);""")


In [ ]:
trans_ts

In [100]:
customers_stats = pysqldf("""SELECT Customer_ID,
count(*) trans_count, 
count(distinct item_id) number_of_distinct_items,
sum(Daily_Revenue) total_revenue ,
sum(Daily_Revenue)/count(*) average_transaction_value
FROM trans_df 
group by 
Customer_ID;""")

In [ ]:
#ax = sns.pointplot(x="ym", y="trans_count", data=trans_ts,aspect_ratio=2.5)

plt.subplot(3,2,1)
ax1=sns.pointplot(x="year_month", y="revenue", data=trans_ts)
plt.subplot(3,2,2)
ax2=sns.pointplot(x="year_month", y="trans_count", data=trans_ts)
plt.subplot(3,2,3)
ax3=sns.pointplot(x="year_month", y="number_of_distinct_items", data=trans_ts)
for item in ax1.get_xticklabels():
    item.set_rotation(90)
for item in ax2.get_xticklabels():
    item.set_rotation(90)
for item in ax3.get_xticklabels():
    item.set_rotation(90)
plt.subplots_adjust(hspace=2.4)
plt.show()

In [ ]:
pysqldf("SELECT * FROM meat LIMIT 10;").head()

In [121]:
from fuzzywuzzy import fuzz,process

custnames_df = pd.DataFrame(trans_df["Customer_Name"].unique(),columns=["customer_name"])
hospitals_df = pd.read_csv("../web_data/Hospital_General_Information.csv")
hospitals_df["Hospital Name"]

0                           MARSHALL MEDICAL CENTER SOUTH
1                          DEKALB REGIONAL MEDICAL CENTER
2                                CRESTWOOD MEDICAL CENTER
3                                CHOCTAW GENERAL HOSPITAL
4                      YUKON KUSKOKWIM DELTA REG HOSPITAL
5                          NORTON SOUND REGIONAL HOSPITAL
6                                      KANAKANAK HOSPITAL
7                                  MANIILAQ HEALTH CENTER
8                           FORT DEFIANCE INDIAN HOSPITAL
9              TUBA CITY REGIONAL HEALTH CARE CORPORATION
10                                         SELLS HOSPITAL
11                           SAN CARLOS APACHE HEALTHCARE
12              CHINLE COMPREHENSIVE HEALTH CARE FACILITY
13                                HOPI HEALTH CARE CENTER
14                                 SAGE MEMORIAL HOSPITAL
15                               CHI-ST VINCENT INFIRMARY
16                      SPARKS MEDICAL CENTER - VAN BUREN
17            

In [126]:
custnames_df["fuzzy_name"] = hospitals_df["Hospital Name"].apply(lambda x : process.extractOne(str(x), trans_df["Customer_Name"].unique()))

In [128]:
custnames_df

,customer_name,fuzzy_name
0,Doctors Hospital at Renaissance,(The University of Texas MD Anderson Cancer Ce...
1,Harris County Hospital District d/b/a Harris H...,"(Adena Regional Medical Center, 92)"
2,The University of Texas MD Anderson Cancer Center,(The University of Texas MD Anderson Cancer Ce...
3,UT Southwestern Zale Lipshy University Hospital,(Harris County Hospital District d/b/a Harris ...
4,Mission Regional Medical Center,(Harris County Hospital District d/b/a Harris ...
5,OBSOLETE - Winchester Hospital Pharmacy,(Harris County Hospital District d/b/a Harris ...
6,Mayo Clinic Eau Claire Hospital Inpatient Phar...,"(Doctors Hospital at Renaissance, 86)"
7,Mayo Clinic Florida,(Harris County Hospital District d/b/a Harris ...
8,Mayo Clinic Red Cedar Hospital,(Harris County Hospital District d/b/a Harris ...
9,Mayo Clinic Rochester St. Marys Campus Pharmacy,"(UMC Health System, 86)"
